## Hello Braket!

Wondering how to access Aquila? Say hello to Braket! 

The AWS Braket interface allows you to send tasks from your laptop to quantum processors via the AWS cloud infrastructure. This tutorial will give you a first taste of what this workflow looks like. It's simple: Define the arrangement of atoms you want to use and specify your Hamiltonian of choice to govern the behavior of those atoms. Submit the program and analyse the results. 

Ready to explore what's possible with Aquila? Let's dive straight in!


### Goal

To say hello to the world of neutral atoms, let's make use of the [Rydberg blockade](https://queracomputing.github.io/Bloqade.jl/dev/tutorials/1.blockade/main/) - the phenomenon at the heart of our quantum computing architecture. We will use it to create ordered phases of matter in one- and two-dimensional arrays of atoms.

### Register

Let's start off with the 1D case.   To achieve our goal, we first need to define our quantum computer's register, consisting of the atoms. We arrange them in a line with a separation of 6.1μm between each pair of atoms. 

In [ ]:
from braket.ahs.atom_arrangement import AtomArrangement
import numpy as np
from utils import show_register

a = 6.1e-6  # meters
N_atoms = 11

register = AtomArrangement()
for i in range(N_atoms):
    register.add([i*a, 0.0])

show_register(register)

### Hamiltonian

The next component we need to specify is the Hamiltonian. It's the energy function that governs the behaviour of our atoms, including their interactions. 

In the lab, the Hamiltonian is implemented by applying lasers to the atoms. 

In [ ]:
from braket.ahs.hamiltonian import Hamiltonian

H = Hamiltonian()

Specifically, the Hamiltonian governing our system of atoms takes the following form:

\begin{align*}
H(t) = \frac{\Omega(t)}{2} \sum_j (e^{i \phi(t)}\ket{g_j}\bra{r_j} +  e^{-i \phi(t)}\ket{r_j}\bra{g_j}) - \sum_j \Delta(t)\hat{n}_j + \sum_{j<k} V_{jk}\hat{n}_j\hat{n}_k,
\end{align*}

where $\Omega$, $\phi$ and $\Delta$ denote the Rabi frequency, laser phase and the detuning of the driving laser field coupling the ground states $\ket{g_j}$ and excited Rydberg state $\ket{r_j}$ of the j-th atom. Our task is to specify these potentially time-dependent Hamiltonian parameters ($\Omega$, $\phi$ and $\Delta$). In case you're new to the Rydberg Hamiltonian, feel free to read up on the physics background on our open source platform [Bloqade](https://queracomputing.github.io/Bloqade.jl/dev/tutorials/1.blockade/main/).


In our current case, we only want to tell the system to activate the set of lasers controlling the Rabi frequency and the global detuning. In particular, we will choose a constant Rabi frequency of $\Omega=2.5\times10^6$ rad/s and a linearly increasing detuning from $\Delta = -9\times10^6$ rad/s to $\Delta = 7\times10^6$ rad/s, together with a total time duration of $t = 4$ μs. The phase parameter will simply be set to zero. The rest of the code simply ensures that $\Omega$ and $\Delta$ are ramped on and off in a way compatible with the experimental setup.

In [ ]:
from utils import show_global_drive, get_drive

omega_min = 0       
omega_max = 2.5e6
detuning_min = -9e6
detuning_max = 7e6

time_max = 4e-6
time_ramp = 0.15*time_max

time_points = [0, time_ramp, time_max - time_ramp, time_max]
omega_values = [amplitude_min, amplitude_max, amplitude_max, amplitude_min]
detuning_values = [detuning_min, detuning_min, detuning_max, detuning_max]
phase_values = [0, 0, 0, 0]

drive = get_drive(time_points, amplitude_values, detuning_values, phase_values)
H += drive

We can plot the waveforms of these driving fields to make sure that they are correctly specified.

In [ ]:
show_global_drive(drive)

However, that's not the full story. These drive terms alone don't tell us how the atoms interact. But interactions are at the core of many-body quantum physics! That's where the Rydberg blockade comes in. The key idea is that within a certain distance - the so-called blockade radius - only one atom will be excited into an $\ket{r}$ state. Having more excitations will cost too much energy. This means that the states of two neighboring atoms will depend on each other and create specific patterns which we will plot below. 

<div class="alert alert-block alert-info">
    <b>Note: </b> In the current version of the AHS module in Braket, this Rydberg interaction term is automatically calculated from the atom positions. Hence, we don't need to worry about specifying it explicitly. Our Hamiltonian is all set!
</div>

### Defining the program (1D case)

Now, we can combine the register and Hamiltonian into a program. In particular, this program falls within the class of Analog Hamiltonian Simulation (AHS). If you're curious about other types of quantum computing, take a look at [gate-based circuits](https://github.com/aws/amazon-braket-examples/blob/main/examples/getting_started/1_Running_quantum_circuits_on_simulators/1_Running_quantum_circuits_on_simulators.ipynb) or tutorials on [quantum annealing](https://github.com/aws/amazon-braket-examples/blob/main/examples/quantum_annealing/Dwave_TravelingSalesmanProblem/Dwave_TravelingSalesmanProblem.ipynb).

In [ ]:
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation

ahs_program = AnalogHamiltonianSimulation(
    register=register, 
    hamiltonian=H
)

### Simulation on your laptop

Before submitting the task to run on actual quantum hardware, let's first check our program by running a local simulation on your own laptop.

In [ ]:
from braket.devices import LocalSimulator

device = LocalSimulator("braket_ahs")

nshots = 1000
task = device.run(ahs_program, shots=n_shots)

# The result can be downloaded directly into an object in the python session:
result = task.result()

What phase of matter did we create? Let's have a look at the average atom density.

In [ ]:
from utils import show_final_avg_density

show_final_avg_density(result)

insert comment on plot along lines of

"We see that the average Rydberg density approximately form a pattern, and the discrepancy can be attributed to several factors, including the finite size of the system and nonadiabaticity throughout the evolution. We expect that as one increase the system size and the duration of the AHS program, the final Rydberg density will approach the ideal pattern." 

### Simulation on Aquila
And now for the truly exciting part! Let's bring Aquila into the game.
<div class="alert alert-block alert-info">
    <b>Note: </b> Running this program on the Aquila processor will incur a cost of 0.30 USD for submitting the program and 0.0X USD per shot. To run this example as is, will cost you Y USD.
</div>

In [ ]:
from braket.aws import AwsDevice 

device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/quera/Aquila")

# To make the program compatible with the quantum hardware, we still need to slice it into discrete time steps:
discretized_ahs_program = ahs_program.discretize(device)

task = device.run(discretized_ahs_program, shots=n_shots)
result = task.result()
show_final_avg_density(result)

![1D_dummyplot](1D.png)

insert comment along lines of:

Local simulation matches quantum reality!

### 2D case

Now that we are familiar with the basic workflow, we can follow the same steps to extend our investigation to the 2D case. 

In [ ]:
b = 6.7e-6  # meters
N_x = N_y = 3

register_2D = AtomArrangement()
for i in range(N_x):
    for j in range(N_y):
    register_2D.add([i*b, j * b])

show_register(register_2D)

In [ ]:
H_2D = Hamiltonian()

omega_min = 0       
omega_max = 2.5e6
detuning_min = -8.75e6
detuning_max = 8.75e6

time_max = 3e-6
time_ramp = 0.25e-6

time_points = [0, time_ramp, time_max - time_ramp, time_max]
omega_values = [amplitude_min, amplitude_max, amplitude_max, amplitude_min]
detuning_values = [detuning_min, detuning_min, detuning_max, detuning_max]
phase_values = [0, 0, 0, 0]

drive_2D = get_drive(time_points, amplitude_values, detuning_values, phase_values)
H_2D += drive_2D

Again, we first run a check on our local, classical laptop: 

In [ ]:
ahs_program_2D = AnalogHamiltonianSimulation(
    register=register_2D, 
    hamiltonian=H_2D
)

device = LocalSimulator("braket_ahs")
result_2D = device.run(ahs_program_2D, shots=n_shots).result()
show_final_avg_density(result_2D)

![2D_dummyplot](2D.png)

Will we see the same behavior on Aquila?

In [ ]:
device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/quera/Aquila")
discretized_ahs_program_2D = ahs_program_2D.discretize(device)

result_2D = device.run(discretized_ahs_program_2D, shots=n_shots).result()
show_final_avg_density(result_2D)

Again, our prediction of a checkerboard phase is confirmed! ...

### Connecting to Braket

Now that you've understood the basic workflow of using the Braket SDK you are ready to start submitting your own tasks to Aquila for real. Before heading over to the [full Braket documentation page](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html), here's an outline of the five most important steps you will be taken through there:

1. [Create an AWS account](https://aws.amazon.com/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all) for free.
2. [Open the Braket console](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fus-east-1.console.aws.amazon.com%2Fbraket%3FhashArgs%3D%2523%26isauthcode%3Dtrue%26region%3Dus-east-1%26state%3DhashArgsFromTB_us-east-1_20274c425666f960&client_id=arn%3Aaws%3Asignin%3A%3A%3Aconsole%2Faqx&forceMobileApp=0&code_challenge=9RCx9puSjDUb1ybh7QD_6ISdRhdFrgRQzISNvKb5oDA&code_challenge_method=SHA-256).
3. Head to the Permissions & Settings subpage to [enable Braket](https://docs.aws.amazon.com/braket/latest/developerguide/braket-enable-overview.html). 
4. Stay on the Permissions & Settings subpage to [enable third-party devices](https://docs.aws.amazon.com/braket/latest/developerguide/braket-enable-third-party.html) (such as QuEra's Aquila).
5. Get started on [your first notebook](https://docs.aws.amazon.com/braket/latest/developerguide/braket-get-started-create-notebook.html!) (If you copy-pasted the code above into a Jupyter notebook on your own GitHub account, you can start by cloning it into the AWS notebook instance.)

Aquila is at your fingertips!